In [1]:
import numpy as np
import cv2, os, shutil, glob, random, json
from PIL import Image
from cv2 import imread, imwrite
# from skimage.morphology import skeletonize, skeletonize_3d
# import helper as hp
# import geopandas as gpd
# import geojson as gjs
# import shapely, random
import pandas as pd
# from shapely.geometry import *
from PIL import Image
import PIL

# from helper import *

In [2]:
def addCol(df):
    df.columns = ['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'label']
    return df

In [3]:
path_e = '/media/hdd/cesar_workspace/keras-retinanet-master/keras-retinanet-master/csv_data/Lanes/'
all_lanes = path_e + 'all_labels_20k.txt'
df = pd.read_csv(all_lanes)
# df.columns = ['filename', 'xmin', 'ymin', 'xmax', 'ymax', 'label']
df = addCol(df)
df.head()

,filename,xmin,ymin,xmax,ymax,label
0,/media/hdd/cesar_workspace/keras-retinanet-mas...,264,401,344,433,Go_straight_turn_right
1,/media/hdd/cesar_workspace/keras-retinanet-mas...,185,481,243,512,Speed_limit
2,/media/hdd/cesar_workspace/keras-retinanet-mas...,414,190,495,227,Turn_left_go_straight
3,/media/hdd/cesar_workspace/keras-retinanet-mas...,438,142,512,174,Go_straight_turn_right
4,/media/hdd/cesar_workspace/keras-retinanet-mas...,482,172,512,299,U-turn-left_turn


In [5]:
path_e = '/media/hdd/cesar_workspace/keras-retinanet-master/keras-retinanet-master/csv_data/Lanes/'
train_e = path_e + 'train_lanes_20k.txt'
val_e = path_e + 'val_lanes_20k.txt'

train_files = pd.read_csv(train_e)
val_files = pd.read_csv(val_e)
train_files = addCol(train_files)
val_v2 = addCol(val_files)
len(train_files), len(val_files)


(45737, 2737)

In [6]:
train_files.head()

,filename,xmin,ymin,xmax,ymax,label
0,/media/hdd/cesar_workspace/keras-retinanet-mas...,264,401,344,433,Go_straight_turn_right
1,/media/hdd/cesar_workspace/keras-retinanet-mas...,185,481,243,512,Speed_limit
2,/media/hdd/cesar_workspace/keras-retinanet-mas...,414,190,495,227,Turn_left_go_straight
3,/media/hdd/cesar_workspace/keras-retinanet-mas...,438,142,512,174,Go_straight_turn_right
4,/media/hdd/cesar_workspace/keras-retinanet-mas...,482,172,512,299,U-turn-left_turn


In [7]:
dic_marks = {'0': 'No_right_turn',
'1':'No_left_turn',
'2':'Prohibited_straight',
'3':'Straight_left_turn_prohibited',
'4':'Go_straight_not_turn_right',
'5':'Left_right_turn_prohibited',
'6':'No_U-turn',
'7':'Speed_limit',
'8':'Speed_limit_child_zone',
'9':'Concession',
'10':'Crosswalk',
'11':'Bike_Priority_Road',
'12':'Turn_left',
'13':'Go_straight',
'14':'Turn_right',
'15':'Turn_left_go_straight',
'16':'Go_straight_turn_right',
'17':'U-turn',
'18':'U-turn-left_turn',
'19':'Unprotected_Left_Turn'}

cat_ids = {label:key for key, label in dic_marks.items()}

In [9]:
all_names = {}
val_names, train_names = list(val_files.filename.unique()) , list(train_files.filename.unique())
names = val_names + train_names
len(names)
for i, name in enumerate(names):
    all_names[name] = i

In [11]:
i

17404

In [10]:
height, width = 512.0, 512.0


In [17]:

# base = '/keras-retinanet-master/keras-retinanet-master/road_mark_detection_dataset/'  
base = 'keras-retinanet-master/keras-retinanet-master/road_mark_detection_dataset/BASEMAP_'

def generate_coco_style(curr_file, name_file):
    json_file = []
    json_dict = {}
    instance = {"image_id":'', "filename":"", "height":"", "width":"", "annotations":[]}
    anno = {"bbox": [], "bbox_mode": 1, "category_id":""}

    for name in name_file:
        id_ = all_names[name]
        frames = curr_file[curr_file.filename == name]

        new_name = name.replace('jpeg', 'png')
#         new_name 
        if not os.path.isfile(new_name):
            print(new_name)
            continue
        instance = {"image_id":id_, "file_name": (new_name),"height":height, "width":width, "annotations":[]} 
        for i,frame in frames.iterrows():
            xmin, ymin, xmax, ymax, label = frame[1:]
            if xmin != ',':
                xmin, ymin, xmax, ymax = map(float, (xmin, ymin, xmax, ymax))
                w, h = (xmax-xmin), (ymax-ymin)
                bbox = [xmin, ymin, w, h]
                _label = cat_ids[label]
            else:
                continue
            anno = {"bbox": bbox, "bbox_mode": 1, "category_id": _label}
            instance["annotations"].append(anno)
        json_file.append(instance)
    return json_file


In [18]:
train_json_file = generat_coco_style(train_files, train_names)
val_json_file = generate_coco_style(val_files, val_names)

In [19]:

train_custom = './dataset/train.json'
val_custom = './dataset/val.json'

def save_custom(json_file, path_to_save):
    with open(path_to_save, 'w') as fp:
        json.dump(json_file, fp)
        
        

In [ ]:
save_custom(train_json_file, train_custom)
save_custom(val_json_file, val_custom)